<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Ground-Truth-Shot-Boundaries" data-toc-modified-id="Ground-Truth-Shot-Boundaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Ground Truth Shot Boundaries</a></span></li></ul></div>

In [ ]:
from esper.prelude import *
from query.models import Tag, Labeler, Frame

# Ground Truth Shot Boundaries
This is a notebook for annotating ground truth shot boundaries for a small subset of frames.

In [ ]:
# Pick the Video ID, start time, and end time
video_id = 123
start_time_seconds = 600
end_time_seconds = 900

We will display all the frames between start and end. You must scroll through and select frames that start new shots. First click "Disable Jupyter keyboard," then hover over the frame. F to expand the thumbnail, S to select the frame. P to play starting from that clip

In [ ]:
video = Video.objects.filter(id=video_id)[0]
frames = range(int(video.fps * start_time_seconds), int(video.fps * end_time_seconds))
def frame_result(video_id, frame_nums):
    from esper.stdlib import simple_result
    materialized_result = []
    for frame_num in frame_nums:
        materialized_result.append({
            'video': video_id,
            'min_frame': frame_num,
            'objects': []
        })
    return simple_result(materialized_result, 'frames')
widget = esper_widget(frame_result(video_id, frames))
widget

In [ ]:
selected_frames = widget.selected

In [ ]:
# visualize your selected frames. We recommend watching the video and counting the number of shots to verify.
print("num boundaries:", len(selected_frames))
selected_frame_nums = [int(video.fps * start_time_seconds) + idx for idx in selected_frames]
esper_widget(frame_result(video_id, selected_frame_nums))

In [ ]:
# Convert boundaries into shots
shots = []
for i in range(0, len(selected_frame_nums) - 1):
    shots.append((selected_frame_nums[i], selected_frame_nums[i+1] - 1))

In [ ]:
# Put shots into database
LABELER, _ = Labeler.objects.get_or_create(name='shot-manual')
LABELED_TAG, _ = Tag.objects.get_or_create(name='shot-manual:labeled')

In [ ]:
new_shots = [
    Shot(min_frame = start_frame,
        max_frame = end_frame,
        video=video,
        labeler=LABELER)
for start_frame, end_frame in shots]

In [ ]:
Shot.objects.bulk_create(new_shots)